In [1]:
# get baseline model
import torch
!pip install transformers
import transformers
from transformers import RobertaForSequenceClassification, RobertaTokenizer
# base_model_path = "/content/drive/MyDrive/CSC3160/GPT-Detection/Baseline/best_model.pt"
# def load_model(model_path):
#     # Load the tokenizer and model from the "roberta-base" pre-trained model
#     tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
#     model = RobertaForSequenceClassification.from_pretrained("roberta-base").cuda()

#     # Load the saved state dict of the fine-tuned model
#     model.load_state_dict(torch.load(model_path))

#     return tokenizer, model
# base_tokenizer,base_model = load_model(base_model_path)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [2]:
Text = "\n\nAs a sophomore in high school, I harboured a fervent desire to throw my hat into the ring for the student council president position, confident in my abilities honed from being an active council participant. Tragically, my hopes and aspirations were tampered by deceitful rumors and vandalized posters disseminated by a rival competitor- sour incidents which left me crestfallen and defeated.\n\nInitially, I was overwhelmed by a wave of despondency. I had signalled considerable zeal and assiduity to the campaign and its outcome yet the universe seemed intent on dealing me a harsh blow. However, like a Phoenix reborn from the ashes, my ordeal galvanized unyielding reflections the essence of which unveiled invaluable life lessons. Above all, I gleaned the notion that the existence of antagonizing forces shouldn't diminish the potency of my ambition. More importantly, I discovered that leadership goes beyond postulations of titles and credentials. Revealingly, I remained a well-engaged council member, steadfastly championing worthy causes that resonated with my person.\n\nThe crowning lesson that emerged from my ordeal was gravity's rapacious generational appetite for failure, not being modestly egalitarian in its distribution. The experience in its entirety had gifted me a dimension of emotional depth that defied age, as I realised that my set back was not an expiration of my ambition or an indictment of my leadership capabilities; but a springboard into greater things. Now, with even more conviction, I am resolved in putting one foot in front of the other on the path to my goals. In summation, though my student council campaign was challenging, it proved to be a crucible of growth that refined my leadership capacity."

In [3]:
inputs = tokenizer(Text, return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

In [6]:
outputs.hidden_states

In [ ]:
def get_rank(inputs, outputs):
    # input is a tensor of shape (1,n), n is the number of words in the input, each element is the index of the word in the vocabulary
    # output.logits is a tensor of shape (1, n, vocab_size), each element is the logit of the word in the vocabulary
    # we need to get the rank of input id in the output.logits
    # output is a tensor of shape (1, n), n is the number of words in the input
    # get the rank of each real word in the prediction of gpt2 model
    length = inputs.shape[1]
    rank = torch.zeros(length)
    logits = outputs.logits
    for i in range(length):
        word = inputs[0][i]
        logit = logits[0][i]
        rank[i] = torch.sum(logit > logit[word])
    return rank

In [ ]:
rank = get_rank(inputs["input_ids"], outputs)

In [ ]:
#Get single sample
Text = "\n\nAs a sophomore in high school, I harboured a fervent desire to throw my hat into the ring for the student council president position, confident in my abilities honed from being an active council participant. Tragically, my hopes and aspirations were tampered by deceitful rumors and vandalized posters disseminated by a rival competitor- sour incidents which left me crestfallen and defeated.\n\nInitially, I was overwhelmed by a wave of despondency. I had signalled considerable zeal and assiduity to the campaign and its outcome yet the universe seemed intent on dealing me a harsh blow. However, like a Phoenix reborn from the ashes, my ordeal galvanized unyielding reflections the essence of which unveiled invaluable life lessons. Above all, I gleaned the notion that the existence of antagonizing forces shouldn't diminish the potency of my ambition. More importantly, I discovered that leadership goes beyond postulations of titles and credentials. Revealingly, I remained a well-engaged council member, steadfastly championing worthy causes that resonated with my person.\n\nThe crowning lesson that emerged from my ordeal was gravity's rapacious generational appetite for failure, not being modestly egalitarian in its distribution. The experience in its entirety had gifted me a dimension of emotional depth that defied age, as I realised that my set back was not an expiration of my ambition or an indictment of my leadership capabilities; but a springboard into greater things. Now, with even more conviction, I am resolved in putting one foot in front of the other on the path to my goals. In summation, though my student council campaign was challenging, it proved to be a crucible of growth that refined my leadership capacity."
label = 1

In [ ]:
# Test on a single sample
import numpy as np
def preprocess_text(tokenizer, input_text, max_length):
    # Tokenize the input text using the tokenizer
    inputs = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,
        return_tensors="pt",
        max_length=max_length,
        truncation=True,
    )

    # Get the input_ids and attention_mask tensors
    return inputs["input_ids"].cuda(), inputs["attention_mask"].cuda()

def get_prediction(model, input_ids, attention_mask):
    # Get the predicted label using the input_ids and attention_mask
    outputs = model(input_ids, attention_mask=attention_mask)
    predicted_label = np.argmax(outputs.logits.detach().cpu().numpy())
    return predicted_label



In [ ]:
input_ids, attention_mask = preprocess_text(
                base_tokenizer, Text, max_length=512
            )
predicted_label = get_prediction(base_model, input_ids, attention_mask)
if predicted_label == 0:
  print("human")
else:
  print("chatGPT")

chatGPT


In [ ]:
# Get Test data
file_path = "/content/drive/MyDrive/CSC3160/GPT-Detection/Baseline/Essay_data/human_data/TOEFL_real_91/data.json"

In [ ]:
import json
def get_data(file_path,is_gpt = True):
  with open(file_path,"r") as f:
    data = json.load(f)
    examples = []
    for example in data:
      text = example["document"]
      label = is_gpt
    examples.append({"text":text,"label":label})
  return examples



In [ ]:
def evaluate_model(file_path, tokenizer = base_tokenizer, model = base_model, is_gpt = True):
  result = []
  dataset = get_data(file_path,is_gpt)
  for data in dataset:
    input_ids, attention_mask = preprocess_text(
                tokenizer, data["text"], max_length=512
            )
    predicted_label = get_prediction(model, input_ids, attention_mask)
    result.append(predicted_label)
  if not is_gpt:
    accuracy = 1-sum(result)/len(result)
  else:
    accuracy = sum(result)/len(result)
  return accuracy




In [ ]:
accuracy = evaluate_model(file_path,is_gpt = False)
print(f"Accuracy at{file_path}:\n {accuracy:.4f}")

Accuracy at/content/drive/MyDrive/CSC3160/GPT-Detection/Baseline/Essay_data/human_data/TOEFL_real_91/data.json:
 0.0000
